In [20]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [21]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '0022adfdd2cc3af510e92a44296127df', 'Date': 'Wed, 21 Aug 2024 21:40:43 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [22]:
import json
data = json.load(open("reviews.json"))
data['reviews']


[{'professor': 'Dr. Emily Chen',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Dr. Chen is an exceptional instructor. Her lectures are engaging and she explains complex concepts clearly. Always available during office hours.'},
 {'professor': 'Prof. Michael Johnson',
  'subject': 'Physics',
  'stars': 4,
  'review': 'Challenging course, but Prof. Johnson makes it interesting. His enthusiasm for the subject is contagious.'},
 {'professor': 'Dr. Sarah Thompson',
  'subject': 'English Literature',
  'stars': 5,
  'review': "Dr. Thompson's passion for literature is evident in every lecture. She encourages critical thinking and fosters great discussions."},
 {'professor': 'Prof. David Lee',
  'subject': 'Economics',
  'stars': 3,
  'review': 'Prof. Lee knows his stuff, but his lectures can be dry. Exams are fair, though.'},
 {'professor': 'Dr. Rachel Goldstein',
  'subject': 'Biology',
  'stars': 5,
  'review': "Best professor I've had at Berkeley! Dr. Goldstein makes complex 

In [23]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [24]:
processed_data[0]

{'values': [-0.02623446,
  0.0010517759,
  0.007107082,
  0.048659205,
  0.032500777,
  0.006167791,
  0.0026667977,
  0.061218116,
  -0.029505556,
  0.0060101477,
  0.013609865,
  0.0041709766,
  0.0075406004,
  -0.020979686,
  0.025695844,
  0.049184684,
  -0.025170367,
  -0.004075734,
  0.0179976,
  0.02559075,
  0.017839957,
  -0.020020688,
  0.05717194,
  0.0018736554,
  -0.053546146,
  -0.052285,
  0.018785818,
  0.0027472614,
  0.01238813,
  0.008880568,
  0.06095538,
  -0.023777852,
  -0.008762335,
  -0.025682708,
  -0.06720856,
  0.089541346,
  -0.008085784,
  0.021465752,
  0.009498004,
  -0.034944247,
  0.017669177,
  0.057855062,
  -0.025722118,
  0.04251112,
  0.031134535,
  -0.022963362,
  -0.026602292,
  0.01975795,
  0.024566067,
  0.013478495,
  -0.06016716,
  0.0067786584,
  0.033288993,
  -0.016381757,
  -0.055385318,
  0.03389329,
  0.026103089,
  0.060377352,
  0.02511782,
  -0.02250357,
  0.0604299,
  -0.044849496,
  -0.012657437,
  0.008387933,
  -0.043167967,
  

In [25]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [26]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}